# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [38]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [39]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [40]:
# 📦 Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Modelos Ensemble
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


**Perform Train Test Split**

In [41]:
# Separar columnas por tipo
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Revisar columnas
print("Categóricas:", categorical_cols)
print("Numéricas:", numerical_cols)


Categóricas: []
Numéricas: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


In [42]:
spaceship.drop(columns=[col for col in ['Name', 'Cabin'] if col in spaceship.columns], inplace=True)


# Convertir la columna 'Transported' a valores binarios
spaceship['Transported'] = spaceship['Transported'].astype(int)

# Separar variables
X = spaceship.drop('Transported', axis=1)
y = spaceship['Transported']


In [43]:
print(spaceship.columns)


Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported'],
      dtype='object')


In [44]:
# Imputación y codificación para categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder',  pd.get_dummies)
])

# Escalado e imputación para numéricas
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])



In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
spaceship = pd.get_dummies(spaceship, drop_first=True)


In [47]:
bagging_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, random_state=42))
])

bagging_pipeline.fit(X_train, y_train)
y_pred_bagging = bagging_pipeline.predict(X_test)

print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))



Bagging Accuracy: 0.7671075330649799


- Random Forests

In [48]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))


Random Forest Accuracy: 0.7676825761932144


In [49]:
gb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
])

gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_test)

print("Gradient Boosting Accuracy:", accuracy_score(y_test, y_pred_gb))


Gradient Boosting Accuracy: 0.7791834387579069


In [ ]:
ada_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', AdaBoostClassifier(n_estimators=100, random_state=42))
])

ada_pipeline.fit(X_train, y_train)
y_pred_ada = ada_pipeline.predict(X_test)

print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ada))


AdaBoost Accuracy: 0.7763082231167338


In [ ]:
print("Bagging:", accuracy_score(y_test, y_pred_bagging))
print("Random Forest:", accuracy_score(y_test, y_pred_rf))
print("Gradient Boosting:", accuracy_score(y_test, y_pred_gb))
print("AdaBoost:", accuracy_score(y_test, y_pred_ada))


Bagging: 0.7740080506037953
Random Forest: 0.7745830937320299
Gradient Boosting: 0.7814836112708453
AdaBoost: 0.7763082231167338


In [55]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score


url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv"
spaceship = pd.read_csv(url)


spaceship.drop(['PassengerId', 'Name', 'Cabin'], axis=1, inplace=True)


spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0], inplace=True)
spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0], inplace=True)
spaceship['Destination'].fillna(spaceship['Destination'].mode()[0], inplace=True)
spaceship['Age'].fillna(spaceship['Age'].median(), inplace=True)
spaceship['VIP'].fillna(spaceship['VIP'].mode()[0], inplace=True)
spaceship['RoomService'].fillna(0, inplace=True)
spaceship['FoodCourt'].fillna(0, inplace=True)
spaceship['ShoppingMall'].fillna(0, inplace=True)
spaceship['Spa'].fillna(0, inplace=True)
spaceship['VRDeck'].fillna(0, inplace=True)


spaceship = pd.get_dummies(spaceship, drop_first=True)


X = spaceship.drop('Transported', axis=1)
y = spaceship['Transported'].astype(int)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


bagging = BaggingClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
gradient_boost = GradientBoostingClassifier(random_state=42)
ada_boost = AdaBoostClassifier(random_state=42)


models = {
    "Bagging": bagging,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boost,
    "AdaBoost": ada_boost
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    results[name] = acc


results_sorted = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
results_sorted


C:\Users\Dixon\AppData\Local\Temp\ipykernel_125516\1044219883.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0], inplace=True)
C:\Users\Dixon\AppData\Local\Temp\ipykernel_125516\1044219883.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

{'Gradient Boosting': 0.7837837837837838,
 'Random Forest': 0.7797584818861415,
 'AdaBoost': 0.7671075330649799,
 'Bagging': 0.7665324899367453}